In [ ]:
% pip install beautifulsoup4
% pip install lxml

In [17]:
from bs4 import BeautifulSoup
import requests
import re

url = "https://www.imensa.de/{city}/{mensa}/{day}.html"
non_digit = re.compile('[^0-9,.]')

In [ ]:
city = ["muenster"]
mensa = [
    "mensa-da-vinci", "bistro-denkpause", "bistro-friesenring",
    "bistro-katholische-hochschule",
    "bistro-durchblick", "bistro-frieden", "bistro-kabu", "bistro-oeconomicum", "hier-und-jetzt", "mensa-am-aasee", "mensa-am-ring", "mensa-bispinghof"
]
days = ["montag", "dienstag", "mittwoch", "donnerstag", "freitag"]

In [40]:
attribute_replacement = {
    "vegan": "Vegan",
    "vegetarisch": "Vegetarian",
    "schwein": "Pork",
    "rind": "Beef",
    "geflügel": "Poultry",

    "farbstoff": "Dyed",
    "konservierungsstoffe": "Preservatives",
    "antioxidationsmittel": "Antioxidants",
    "geschmacksverstärker": "Flavor enhancers",
    "geschwefelt": "sulphurated",
    "geschwärzt": "blackened",
    "gewachst": "waxed",
    "phosphat": "Phosphate",
    "süßungsmittel": "Sweeteners",
    "phenylalaninquelle": "Phenylalanine source",

    "gluten": "Gluten",
    "dinkel": "Spelt",
    "gerste": "Barley",
    "hafer": "Oats",
    "kamut": "Kamut",
    "roggen": "Rye",
    "weizen": "Wheat",

    "krebstiere": "Crustaceans",
    "ei": "Egg",
    "fisch": "Fish",
    "erdnüsse": "Peanuts",
    "soja": "Soy",
    "milch": "Milk",

    "schalenfrüchte": "Nuts",
    "mandeln": "Almonds",
    "haselnüsse": "Hazelnuts",
    "walnüsse": "Walnuts",
    "cashewkerne": "Cashews",
    "pecannüsse": "Pecans",
    "paranüsse": "Brazil nuts",
    "pistazien": "Pistachios",
    "macadamianüsse": "Macadamias",

    "sellerie": "Celery",
    "senf": "Mustard",
    "sesam": "Sesame",
    "lupinen": "Lupines",
    "weichtiere": "Molluscs",
    "schwefeldioxid": "Sulfur dioxide",
}

In [45]:
f = requests.get(url)
document = f.text

In [52]:
soup = BeautifulSoup(document, "lxml")
type(soup)

bs4.BeautifulSoup

In [ ]:
print(soup.prettify())

In [47]:
mensa_rating_avg = float(soup.find(class_="aw-ratings-average").text.replace(",", "."))
mensa_rating_count = int(soup.find(class_="aw-ratings-count").text.replace(".", ""))

print(mensa_rating_count, mensa_rating_avg)

2001 3.8


In [51]:
main_meal = True
for div in soup.find_all(class_="aw-meal-category"):
    for meal in div.find_all(class_="aw-meal row no-margin-xs"):
        name = meal.find(class_="aw-meal-description").text
        price_for_students = meal.find(title="Preis für Studierende").text[:-2].replace(",", ".")

        attributes = meal.find(class_="small aw-meal-attributes").span.text.replace("\xa0", "").split(" ")
        category = attribute_replacement[attributes[0].lower()]

        att_type = 0
        additives = []
        allergies = []

        for att in attributes[1:]:
            att = att.lower()
            replacement = att
            if att in attribute_replacement:
                replacement = attribute_replacement[att]

            if att == "zusatz":
                att_type = 1
            elif att == "allergen":
                att_type = 2
            elif att_type == 1:
                additives.append(replacement)
            elif att_type == 2:
                allergies.append(replacement)

        last_occurrence = meal.find(title="Zuletzt angeboten")
        if last_occurrence:
            last_occurrence = last_occurrence.text

        ratings_count_tag = meal.find(class_="aw-meal-histogram-count")
        ratings_count = 0
        if ratings_count_tag:
            ratings_count = int(non_digit.sub("", ratings_count_tag.text))

        ratings_avg_tag = meal.find(class_="aw-meal-histogram-average")
        ratings_avg = 0
        if ratings_avg_tag:
            ratings_avg = float(non_digit.sub("", ratings_avg_tag.text.replace(",", ".")))

        print(name, price_for_students, main_meal, category, additives, allergies, last_occurrence, ratings_count,
              ratings_avg)
    main_meal = False

Penne mit Gemüsesugo 1.80 True Vegan [] ['Gluten', 'Wheat'] 06.09.2022 2 4.0
Gefüllte Wirsingroulade mit Mett mit Jus vom Schwein 2.30 True Pork [] ['Gluten', 'Celery', 'Wheat'] None 0 0
Spinatknödel mit Gorgonzolasauce 2.05 True Vegetarian [] ['Egg', 'Gluten', 'Milk', 'Wheat'] 11.10.2022 2 4.0
Erbsen und Möhren 0.30 False Vegan [] [] 10.11.2022 6 4.8
Rahmkohlrabi 0.50 False Vegan ['Dyed'] ['Soy'] 03.11.2022 2 4.0
Salzkartoffeln 0.70 False Vegan ['Dyed'] [] 04.11.2022 15 4.5
Beilagensalat mit Essig-Öl-Dressing 0.90 False Vegan ['Antioxidants'] ['Mustard'] gestern 9 3.2
